In [ ]:
# Program to rename all image file


# import os
# os.chdir(r'C:\Users\Admin\OneDrive\Desktop\AlzheimerNet Data\NC\Normal')
# print(os.getcwd())

# #Display number with leading zeros

# for count, f in enumerate(os.listdir()):
# 	f_name, f_ext = os.path.splitext(f)
# 	f_name = "tuberculosis" + str(count).zfill(5)

# 	new_name = f'{f_name}{f_ext}'
# 	os.rename(f, new_name)

In [ ]:
#Program for resize the images

# import PIL
# import os
# import os.path
# from PIL import Image

# f = r'C:\Users\Admin\OneDrive\Desktop\AlzheimerNet Data\NC\Normal'

# for file in os.listdir(f):
#     f_img = f+"/"+file
#     img = Image.open(f_img)
#     img = img.resize((224,224))
#     img.save(f_img)

In [ ]:


#Clahe for image enhancement

# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# import glob
# import time

# input = r'C:\Users\Admin\OneDrive\Desktop\Lung Data\covid'
# i = 0
# #start = time.time()
# for img in glob.glob(input + '/*.jpg'):
#     #Test%04i.png
#     image = cv2.imread(r'C:\Users\Admin\OneDrive\Desktop\AlzheimerNet Data\NC\Normal%05i.jpg'%i,0)
#     #Creating CLAHE 
#     clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(10,10))
#     #Apply CLAHE to the original image
#     image_clahe = clahe.apply(image)
#     cv2.imwrite(r'C:\Users\Admin\OneDrive\Desktop\AlzheimerNet Data\NC\Normal%05i.jpg'%i,image_clahe)
#     i += 1



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
from IPython.display import Image, display
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
import tensorflow as tf 

print("TensorFlow Version:", tf.__version__)

In [ ]:
image_dir = Path('C:/Users/Admin/OneDrive/Desktop/AlzheimerNet Data/')

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df = image_df[image_df['Label'].apply(lambda x: x[-2:] != 'GT')]

In [ ]:
# Shuffle the DataFrame and reset index
image_df = image_df.sample(frac=1).reset_index(drop = True)

# Show the result
image_df.head(50)

In [ ]:
# Display 20 picture of the dataset with their labels
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[i]))
    ax.set_title(image_df.Label[i])
plt.tight_layout()
plt.show()

In [ ]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, train_size=0.6, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(255, 255),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(255, 255),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
# Load the pretained model
pretrained_model = tf.keras.applications.InceptionV3(
    input_shape=(255, 255, 3),
    include_top=False,
    weights='imagenet',
)

pretrained_model.trainable = False
pretrained_model.summary()

In [ ]:
for layer in pretrained_model.layers:
    layer.trainable=False

In [ ]:
# Proposed AlzheimerNet Model
custom_inception_model = Sequential([
        pretrained_model,
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Flatten(),
        Dense(6, activation='softmax')        
    ], name = "inception_cnn_model")

In [ ]:
from tensorflow.python.keras.metrics import Metric


custom_inception_model.compile(optimizer='rmsprop',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=['accuracy'])

custom_inception_model.summary()

In [ ]:
history = custom_inception_model.fit(
          train_images,
          epochs = 300,
          verbose=1,
          validation_data = val_images,
          )

In [ ]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()
#model.summary()

In [ ]:
pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
results = custom_inception_model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
# Predict the label of the test_images
pred = custom_inception_model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f'The first 6 predictions: {pred[:6]}')

In [ ]:
from sklearn.metrics import classification_report
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
# Display 15 picture of the dataset with their labels
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[i]))
    ax.set_title(f"True: {test_df.Label.iloc[i]}\nPredicted: {pred[i]}")
plt.tight_layout()
plt.show()